# BEE 4750 Homework 5: Mixed Integer and Stochastic Programming

**Name**: Jiaming Yuan (jy729), Ari Schor (aes392), Grace Raab (gar238)

**ID**:

> **Due Date**
>
> Thursday, 12/05/24, 9:00pm

## Overview

### Instructions

-   In Problem 1, you will use mixed integer programming to solve a
    waste load allocation problem.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [12]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `c:\Users\grcra\OneDrive\Desktop\FA24\BEE4750\hw5-ari-j`


In [13]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Problems (Total: 30 Points)

### Problem 1 (30 points)

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided below.

-   **LF**: Capacity 200 Mg, fixed cost \$2000/day, tipping cost
    \$50/Mg;
-   **MRF**: Capacity 350 Mg, fixed cost \$1500/day, tipping cost
    \$7/Mg, recycling cost \$40/Mg recycled;
-   **WTE**: Capacity 210 Mg, fixed cost \$2500/day, tipping cost
    \$60/Mg;

The MRF recycling rate is 40%, and the ash fraction of non-recycled
waste is 16% and of recycled waste is 14%. Transportation costs are
\$1.5/Mg-km, and the relative distances between the cities and
facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

**In this problem**:

-   Formulate the waste load allocation problem and implement it in
    `JuMP`.
-   Draw a diagram showing the flows of waste between the cities and the
    facilities. Which facilities (if any) will not be used? Does this
    solution make sense?

In [21]:
waste_model = Model(HiGHS.Optimizer) #initialize model object
#create variables: amount of waste disposed, amount of residual, operational status 
@variable(waste_model, 0 <= W[1:3, 1:3] <= 1.0e10) #W is amount of waste from each city to each disposal plant
@variable(waste_model, 0 <= R[1:3] <= 1.0e10) #R is the residual from MRF and WTE (MRF to LF, MRF to WTE, and WTE to LF)
@variable(waste_model, Y[1:3], Bin) #Y is operational status of each plant

#create objective function (profit function)
@objective(waste_model, Min, 2000*Y[1]+1500*Y[2]+2500*Y[3]+57.5W[1]+72.5*W[2]+69.5*W[3]+68*W[4]+60.5*W[5]+90.5*W[6]+82.5*W[7]+75*W[8]+90*W[9]+98*R[1]+77*R[2]+82.5*R[3])

#contraints for city waste production, capacity of each facility, residuals from MRF and WTE, and committment status
@constraint(waste_model, cityone, W[1,1]+W[1,2]+W[1,3] == 100)
@constraint(waste_model, citytwo, W[2,1]+W[2,2]+W[2,3] == 90)
@constraint(waste_model, citythree, W[3,1]+W[3,2]+W[3,3] == 120)
@constraint(waste_model, lfmax, W[1,1]+W[2,1]+W[3,1]+R[1]+R[2] <= 200)
@constraint(waste_model, mrfmax, W[1,2]+W[2,2]+W[3,2] <= 350)
@constraint(waste_model, wtemax, W[1,3]+W[2,3]+W[3,3]+R[3] <= 210)
@constraint(waste_model, wteres, R[2]-0.16*(W[1,3]+W[2,3]+W[3,3])-0.14*R[3] == 0)
@constraint(waste_model, mrfres, R[1]+R[3]-0.6*(W[1,2]+W[2,2]+W[3,2]) == 0)
@constraint(waste_model, commit1, Y[1] == 1)
@constraint(waste_model, commit2, !Y[2] => {W[1,2] + W[2,2] + W[3,2] == 0})
@constraint(waste_model, commit3, !Y[3] => {W[1,3]+W[2,3]+W[3,3]+R[3] == 0});

In [22]:
optimize!(waste_model)
@show value.(W)
@show value.(R)
@show value.(Y)
@show value.(cityone)
@show value.(citytwo)
@show value.(citythree)
@show value.(lfmax)
@show value.(mrfmax)
@show value.(wtemax);

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-01, 4e+10]
  Cost   [6e+01, 2e+03]
  Bound  [1e+00, 1e+10]
  RHS    [1e+00, 4e+02]
Presolving model
11 rows, 16 cols, 41 nonzeros  0s
8 rows, 13 cols, 35 nonzeros  0s
8 rows, 13 cols, 35 nonzeros  0s

Solving MIP model with:
   8 rows
   13 cols (2 binary, 0 integer, 0 implied int., 11 continuous)
   35 nonzeros
MIP-Timing:     0.00038 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp 

![](4750hw5_figure.jpeg "Waste Allocation Solution")

### Homework 5 Writeup

The problem objective is to minimize the cost for waste disposal from three cities given three disposal plants: landfill, materials recovery facility, and waste-to-energy. To do this, we created 3 arrays of variables where $W = $ amount of waste disposed, $R = $ the residual amount from waste disposal plants, and $Y = $ the operational status of each plant. $W$ is an array with 3 rows and 3 columns, where each row represents one of the 3 cities, and each column represents one of the 3 disposal methods so, for example, the amount of waste from city 1 going to the landfill is found at $W_{1,1}$. $R$ and $Y$ are both 1 by 3 arrays. In all of these variables, we include a non-negativity constraint as well as a maximum value for each of them. $W$ and $R$ cannot exceed ${10}^{10}$ and $Y$ is binary where 0 means the facility is not operating and 1 means a facility is operating.

#### Objective
Now that we have our variables and we know that we need to minimize costs, we have to create an objective function where all of our terms are in units of dollars. For $Y$, this means that we multiply the fixed cost for each plant by the operational status of the plant, since we only pay the fixed cost if the plant is running. Therefore, for the first part of the equation, we have derived $2000Y_1+1500Y_2+2500Y_3$. Next, we have to look at $W$. Here, the total cost takes into account the travel cost to each disposal site as well as the tipping cost for each disposal method, so for city 1 landfill disposal, for example, we get $(50 + 1.5(5))W[1,1]$. We also have to include recycling cost for MRF, so we muliply the recycling rate by the recycling cost. If we do this for each combination of city and plant we get: $(50 + 1.5(5)){W}_{1,1} + (50 + 1.5(15)){W}_{2,1} + (50 + 1.5(13)){W}_{3,1} + (7 + 0.4(40) + 1.5(30)){W}_{1,2} + (7 + 0.4(40) + 1.5(25)){W}_{2,2} + (7 + 0.4(40) + 1.5(45)){W}_{3,2} + (60 + 1.5(15)){W}_{1,3} + (60 + 1.5(10)){W}_{2,3} + (60 + 1.5(20)){W}_{1,3}$ We can simplify this to get $57.5W_1 + 72.5W_2 + 69.5W_3 + 68W_4 + 60.5W_5 + 90.5W_6 + 82.5W_7 + 75W_8 + 90W_9$. Finally, we need to account for the amount of residual waste that needs to be landfilled, which are in the variable $R$. Since only 40% of materials entering MRF can be recycled, that means 0.6*(the amount of waste from each city to MRF) must either go to the WTE or the landfill. $R1$ is the amount of MRF residual going to the WTE and $R3$ is the residual going to the landfill. Finally, $R2$ is the amount of ashes from the WTE that also end up in the landfill. They are accounted in the costs for each plant -- $R3$ and $R2$ are the residual amount that go to the landfill, so those amounts also have the $50 tipping cost. Similarly, the $60 tipping cost for WTE applies to $R1$.

#### Constraints
Once we have our objective function, we need to come up with our constraints. First, we are given the total waste produced by each city so we know that the 3 disposal methods for each city need to sum to the given totals. For city 1 we have ${W}_{1,1}+{W}_{1,2}+{W}_{1,3} = 100$, for city 2 we have ${W}_{2,1}+{W}_{2,2}+{W}_{2,3} = 90$, and for city 3 we have ${W}_{3,1}+{W}_{3,2}+{W}_{3,3} = 120$. We are also given maximum capacities for each disposal method, so we need the total waste disposed of at each plant to be less than or equal to the given quantities. For the landfill we have ${W}_{1,1}+{W}_{2,1}+{W}_{3,1} + R_1 + R_2 \leq 100$, for the materials recycling facility we have ${W}_{1,2}+{W}_{2,2}+{W}_{3,2} \leq 350$, and for the waste-to-energy facility we have ${W}_{1,3}+{W}_{2,3}+{W}_{3,3} + R_3 \leq 210$. There are also two constraints for the amount of residual. Residuals from the MRF are equal to 60% waste input to the MRF, so $R_1+R_3=0.6*(W_{1,2}+W_{2,2}+W_{3,2})$. Then, the residual ashes from the WTE that must go to the landfill equals 16% non-recycled waste and 14% recycled waste, so $R_2=0.16*(W_{1,3}+W_{2,3}+W_{3,3})-0.14*R_3$. We also have committment constraints for each plant. Landfill waste is set to always be in operation with a commitment integer value of $Y_1 = 1$. The MRF plant is set to be operational ($Y_2=1$) only when the waste going toward that plant is nonzero. Similarly, the WTE plant is set to be operational only when the waste distributed to that plant summed with the residual from MRF is nonzero.

#### Results
After running the model, the results are shown in the diagram above. All of City 1's waste goes straight to the landfill and all of City 2's waste goes to the WTE. Most of City 3's waste goes to landfill (79.05 Mg/day) while some goes to WTE (40.95 Mg/day). No waste goes to the MRF, shown by its status $Y_2 = 0$, which makes sense because of the higher cost and farther distance from the three cities. Finally, 20.95 Mg/day of residuals from the WTE also go to the landfill. The results fill the landfill to full capacity and put whatever couldn't be landfilled to the WTE. This makes sense given the lower cost and close proximity to the three cities unlike the large distance of the cities from the MRF facility.

## References

List any external references consulted, including classmates.

#### References:

To resolve domain error in the model, added boundaries to the variables based on a response from this thread on the Julia discourse page: https://discourse.julialang.org/t/indicator-constraints-in-iqp/107587

This StackOverflow thread for inserting the figure image: https://stackoverflow.com/questions/10628262/inserting-image-into-ipython-notebook-markdown

Julia Documentation (12/4 office hours): https://jump.dev/JuMP.jl/stable/manual/variables/#Binary-variables